In [2]:
from quantopian.pipeline import Pipeline

In [3]:
def make_pipeline():
    return Pipeline()

In [4]:
#Create pipeline object
pipe = make_pipeline()

In [5]:
from quantopian.research import run_pipeline

In [6]:
result = run_pipeline(pipeline=pipe, start_date='2017-01-03',end_date='2017-01-03')

In [7]:
result.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8356 entries, (2017-01-03 00:00:00+00:00, Equity(2 [HWM])) to (2017-01-03 00:00:00+00:00, Equity(50569 [OUSM]))
Empty DataFrame

### Factors

In [8]:
from quantopian.pipeline.data.builtin import USEquityPricing

In [9]:
from quantopian.pipeline.factors import SimpleMovingAverage

In [10]:
SimpleMovingAverage(inputs=[USEquityPricing.close], window_length= 30);

In [11]:
def make_pipeline():
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length= 30)
    return Pipeline(columns={'30 Day Mean Close':mean_close_30})

In [12]:
results = run_pipeline(make_pipeline(),'2017-01-03','2017-01-03')

In [13]:
def make_pipeline():

    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5    

    #Use masking to save a bit of computational work
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length= 30, mask=small_price)
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length= 10, mask=small_price)

    
    percent_diff = (mean_close_10 - mean_close_30)/mean_close_30
    
    #Filtering out negative percent differences
    percent_filter = percent_diff > 0
    
    #Combine filters
    final_filter = percent_filter & small_price
    
    return Pipeline(columns={'30 Day Mean Close':mean_close_30 , 
                             'Percent Difference':percent_diff, 
                             'Latest Close':latest_close, 
                             'Percent Filter':percent_filter},
                             screen=final_filter)

In [14]:
results = run_pipeline(make_pipeline(),'2017-01-03','2017-01-03')


In [15]:
from quantopian.pipeline.data import morningstar
from quantopian.pipeline.classifiers.morningstar import Sector

### Classifiers

In [16]:
morningstar_sector = Sector()

In [17]:
exchange = morningstar.share_class_reference.exchange_id.latest

In [18]:
nyse_filter = exchange.eq('NYS')

In [19]:
def make_pipeline():

    latest_close = USEquityPricing.close.latest
    small_price = latest_close < 5   
    
    #Classifier
    nyse_filter = exchange.eq('NYS')

    #Use masking to save a bit of computational work
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length= 30, mask=small_price)
    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length= 10, mask=small_price)

    
    percent_diff = (mean_close_10 - mean_close_30)/mean_close_30
    
    #Filtering out negative percent differences
    percent_filter = percent_diff > 0
    
    #Combine filters
    final_filter = percent_filter & nyse_filter
    
    return Pipeline(columns={'30 Day Mean Close':mean_close_30 , 
                             'Percent Difference':percent_diff, 
                             'Latest Close':latest_close, 
                             'Percent Filter':percent_filter},
                             screen=final_filter)

In [20]:
results = run_pipeline(make_pipeline(),'2017-01-03','2017-01-03')

In [21]:
results.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 66 entries, (2017-01-03 00:00:00+00:00, Equity(2586 [EQS])) to (2017-01-03 00:00:00+00:00, Equity(49678 [OSG]))
Data columns (total 4 columns):
30 Day Mean Close     66 non-null float64
Latest Close          66 non-null float64
Percent Difference    66 non-null float64
Percent Filter        66 non-null bool
dtypes: bool(1), float64(3)
memory usage: 2.1+ KB
